In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pykalman import KalmanFilter
import matplotlib.pyplot as plt

# --- Data Preprocessing and Quarterly Aggregation ---

def convert_to_quarterly(df, date_col='date'):
    """
    Converts the dataframe to quarterly data by aggregating monthly or weekly data.
    """
    df['Quarter'] = df[date_col].dt.to_period('Q')
    df_quarterly = df.groupby('Quarter').agg({
        'Gross_Sales': 'mean',
        'Account_Receivable': 'mean'
    }).reset_index()
    return df_quarterly

def merge_datasets(df1, df2, df3):
    """
    Merges df1 (with ECI codes) and df2 (with ACE IDs) using df3 (mapping between ECI and ACE IDs).
    """
    # Merge df1 with df3 on 'ECI_Code'
    df1_mapped = pd.merge(df1, df3, on='ECI_Code', how='inner')
    
    # Merge the result with df2 on 'ACE_ID' and 'Quarter'
    df_final = pd.merge(df1_mapped, df2, on=['ACE_ID', 'Quarter'], how='outer')
    return df_final

# --- Missing Data Imputation Methods ---

def kalman_filter_imputation(data):
    """
    Applies Kalman Filter to fill missing values.
    """
    kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
    data_filled, _ = kf.em(data, n_iter=5).smooth(data)
    return data_filled.flatten()

def interpolate_missing(df):
    """
    Linear interpolation to fill missing values.
    """
    df['Gross_Sales'] = df['Gross_Sales'].interpolate(method='linear')
    df['Account_Receivable'] = df['Account_Receivable'].interpolate(method='linear')
    return df

def fill_missing_values(df, method='kalman'):
    """
    Fill missing values using different methods: 'kalman', 'interpolate', 'mean', or 'ffill'.
    """
    if method == 'kalman':
        df['Gross_Sales'] = kalman_filter_imputation(df['Gross_Sales'])
        df['Account_Receivable'] = kalman_filter_imputation(df['Account_Receivable'])
    elif method == 'interpolate':
        df = interpolate_missing(df)
    elif method == 'mean':
        df['Gross_Sales'] = df['Gross_Sales'].fillna(df['Gross_Sales'].mean())
        df['Account_Receivable'] = df['Account_Receivable'].fillna(df['Account_Receivable'].mean())
    elif method == 'ffill':
        df['Gross_Sales'] = df['Gross_Sales'].fillna(method='ffill')
        df['Account_Receivable'] = df['Account_Receivable'].fillna(method='ffill')
    
    return df

# --- Forecasting Models ---

def run_linear_regression(X_train, y_train, X_test):
    """
    Runs linear regression and returns the predictions on the test set.
    """
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

def run_random_forest(X_train, y_train, X_test):
    """
    Runs Random Forest Regressor and returns predictions.
    """
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

def run_arima(y_train, order=(1, 1, 1)):
    """
    Runs ARIMA model and returns the predictions on the test set.
    """
    model = ARIMA(y_train, order=order)
    model_fit = model.fit()
    return model_fit

def run_sarima(y_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 4)):
    """
    Runs SARIMA model and returns the fitted model.
    """
    model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit()
    return model_fit

# --- Evaluation & Plotting ---

def evaluate_model(y_test, y_pred):
    """
    Evaluates model performance by calculating RMSE and plotting predictions vs actuals.
    """
    rmse = np.sqrt(((y_pred - y_test) ** 2).mean())
    print(f"RMSE: {rmse}")

    plt.figure(figsize=(10, 5))
    plt.plot(y_test, label='Actual', color='blue')
    plt.plot(y_pred, label='Predicted', color='red')
    plt.legend()
    plt.title('Predictions vs Actuals')
    plt.show()

# --- Main Workflow ---

def main(df1, df2, df3):
    # Step 1: Convert df1 to quarterly
    df1_quarterly = convert_to_quarterly(df1)
    
    # Step 2: Merge df1_quarterly with df2 (Revenue data) using df3 mapping
    df_combined = merge_datasets(df1_quarterly, df2, df3)

    # Step 3: Handle missing values
    df_combined = fill_missing_values(df_combined, method='kalman')

    # Step 4: Prepare data for forecasting (use Gross_Sales and Revenue)
    X = df_combined[['Gross_Sales', 'Account_Receivable']]  # Features
    y = df_combined['Revenue']  # Target variable

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 5: Run different forecasting models and evaluate

    # Linear Regression
    print("Linear Regression Results:")
    y_pred_lr = run_linear_regression(X_train, y_train, X_test)
    evaluate_model(y_test, y_pred_lr)

    # Random Forest
    print("Random Forest Results:")
    y_pred_rf = run_random_forest(X_train, y_train, X_test)
    evaluate_model(y_test, y_pred_rf)

    # ARIMA
    print("ARIMA Results:")
    arima_model = run_arima(y_train)
    y_pred_arima = arima_model.forecast(steps=len(y_test))
    evaluate_model(y_test, y_pred_arima)

    # SARIMA
    print("SARIMA Results:")
    sarima_model = run_sarima(y_train)
    y_pred_sarima = sarima_model.forecast(steps=len(y_test))
    evaluate_model(y_test, y_pred_sarima)

# --- Example Usage ---
# Assuming df1, df2, df3 are loaded DataFrames
# main(df1, df2, df3)
